In [1]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset

import advertorch
from advertorch.attacks import GradientSignAttack
from advertorch.attacks import LinfPGDAttack
from advertorch.attacks import CarliniWagnerL2Attack

# Need the model for which AE is generated here
from Resnet_adaptation import new_cnn_model

In [2]:
# RANDOMIZE
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Generate AEs for the second model
PATH = './lr/best_model_CNN.pt'
model = torch.load(PATH).to(device) if torch.cuda.is_available() else torch.load(PATH, map_location=torch.device('cpu'))

In [14]:
print(model)

ResNet_Adaptation(
  (conv1): Conv2d(1, 64, kernel_size=(8, 8), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.01, inplace=False)
  (dropout2): Dropout(p=0.01, inplace=False)
  (layer1): Sequential(
    (0): block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0

In [5]:
# get scalers
training_set = np.load('./data/train.npz')

mean = round(training_set['x'][...,0].mean(), 2)
std = round(training_set['x'][...,0].std(), 2)

print("Mean ={}, Std = {}".format(mean, std))

Mean =4090.28, Std = 2164.09


In [6]:
# data for which the AEs are to be generated 
test_set = np.load('./data/test.npz')
test_inputs = np.squeeze(test_set['x'])

print(f'shape = {test_inputs.shape}, Max = {np.max(test_inputs)}')

test_inputs = (test_inputs - mean)/std
print(f'shape = {test_inputs.shape}, Max = {np.max(test_inputs)}')

# Use the max and min values to constain the AEs in the box
clip_min = np.min(test_inputs)
clip_max = np.max(test_inputs)

print(f'Box Constraints {clip_min},{clip_max}')

shape = (2616, 12, 150), Max = 11644.0
shape = (2616, 12, 150), Max = 3.4904832978295723
Box Constraints -1.8900692669898203,3.4904832978295723


In [7]:
class TR_dataset_no_targets(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, key):
        input_seq = self.data_list[key]
        return input_seq

In [8]:
BATCH_SIZE = 1
adv_set = TR_dataset_no_targets(test_inputs)

adv_loader = torch.utils.data.DataLoader(dataset=adv_set,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=None,
                                               shuffle=False)

In [9]:
# same rmse loss fn, in PyTorch
def loss_fun(out, tar):
    rmse = torch.sqrt(torch.mean((out - tar)**2))
    return rmse

In [10]:
def generate_fgsm_no_target(model, loss_func, eps, adv_dataloader, cmin, cmax):
    
    FGSM_adv_signals =[]
    eps_value = torch.tensor([eps], dtype = torch.float32).to(device)
    
    adversary = GradientSignAttack(model, 
                                   loss_fn= loss_func,
                                   eps = eps_value, 
                                   clip_min=cmin, 
                                   clip_max=cmax, 
                                   targeted= False)
    
    for bi, data in enumerate(adv_dataloader):
        data_batch = data.float().to(device)
        adv_signal = adversary.perturb(data_batch)
        FGSM_adv_signals.append(np.squeeze(adv_signal.cpu().numpy()))
        
    return FGSM_adv_signals

In [11]:
epsillon = [0., 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.2]

print(epsillon[7])

adv_fgsm = generate_fgsm_no_target(model, loss_fun, epsillon[7], adv_loader, clip_min, clip_max )
adv_fgsm= np.array(adv_fgsm)

print(adv_fgsm.shape)
np.save('./lr/adv_fgsm.npy', adv_fgsm)

0.2
(2616, 12, 150)


In [12]:
def generate_bim_no_target(model, loss_func, eps, adv_dataloader, cmin, cmax):
    
    BIM_adv_signals =[]
    eps_value = torch.tensor([eps], dtype = torch.float32).to(device)
    
    adversary = LinfPGDAttack(model, 
                              loss_fn= loss_func,
                              eps = eps_value,
                              eps_iter=0.05, # per iteration change
                              nb_iter=10, #num_iterations
                              clip_min=cmin, 
                              clip_max=cmax, 
                              targeted= False)
    
    for bi, data in enumerate(adv_dataloader):
        data_batch = data.float().to(device)
        adv_signal = adversary.perturb(data_batch)
        BIM_adv_signals.append(np.squeeze(adv_signal.cpu().numpy()))
        
    return BIM_adv_signals

In [13]:
print(epsillon[7])

adv_bim = generate_bim_no_target(model, loss_fun, epsillon[7], adv_loader, clip_min, clip_max )
adv_bim= np.array(adv_bim)

print(adv_bim.shape)
np.save('./lr/adv_bim.npy', adv_bim)

0.2
(2616, 12, 150)
